In [1]:
#data ingestion

loader = TextLoader("speech.txt")
text_documents = loader.load()
text_documents

[Document(page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not 

In [9]:
# import os
# from dotenv import load_dotenv
# load_dotenv()

# os.environ['OPEN_AI_KEY'] = os.getenv("OPENAI_API_KEY")


In [4]:
#web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

#load chunk and index

loader = WebBaseLoader(web_path=("https://lilianweng.github.io/posts/2020-10-29-odqa/",), bs_kwargs=dict(parse_only = bs4.SoupStrainer(
    class_ = ("post-header", "post-content")
)))

webpagedoc = loader.load()

In [5]:
webpagedoc

[Document(page_content='\n\n      How to Build an Open-Domain Question Answering System?\n    \nDate: October 29, 2020  |  Estimated Reading Time: 33 min  |  Author: Lilian Weng\n\n\n\n[Updated on 2020-11-12: add an example on closed-book factual QA using OpenAI API (beta).\nA model that can answer any question with regard to factual knowledge can lead to many useful and practical applications, such as working as a chatbot or an AI assistant🤖. In this post, we will review several common approaches for building such an open-domain question answering system.\nDisclaimers given so many papers in the wild:\n\nAssume we have access to a powerful pretrained language model.\nWe do not cover how to use structured knowledge base (e.g. Freebase, WikiData) here.\nWe only focus on a single-turn QA instead of a multi-turn conversation style QA.\nWe mostly focus on QA models that contain neural networks, specially Transformer-based language models.\nI admit that I missed a lot of papers with archite

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(webpagedoc)
documents[:5]

[Document(page_content='How to Build an Open-Domain Question Answering System?\n    \nDate: October 29, 2020  |  Estimated Reading Time: 33 min  |  Author: Lilian Weng\n\n\n\n[Updated on 2020-11-12: add an example on closed-book factual QA using OpenAI API (beta).\nA model that can answer any question with regard to factual knowledge can lead to many useful and practical applications, such as working as a chatbot or an AI assistant🤖. In this post, we will review several common approaches for building such an open-domain question answering system.\nDisclaimers given so many papers in the wild:', metadata={'source': 'https://lilianweng.github.io/posts/2020-10-29-odqa/'}),
 Document(page_content='Assume we have access to a powerful pretrained language model.\nWe do not cover how to use structured knowledge base (e.g. Freebase, WikiData) here.\nWe only focus on a single-turn QA instead of a multi-turn conversation style QA.\nWe mostly focus on QA models that contain neural networks, specia

In [10]:
#vector embedding and storage
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(documents[:15], OpenAIEmbeddings(openai_api_key = OPENAPIKEY))

In [11]:
#vector database

query = "What is Open-Domain Question Answering?"
result = db.similarity_search(query)
result[0].page_content

'What is Open-Domain Question Answering?#\nOpen-domain Question Answering (ODQA) is a type of language tasks, asking a model to produce answers to factoid questions in natural language. The true answer is objective, so it is simple to evaluate model performance.\nFor example,\nQuestion: What did Albert Einstein win the Nobel Prize for?\nAnswer: The law of the photoelectric effect.\nThe “open-domain” part refers to the lack of the relevant context for any arbitrarily asked factual question. In the above case, the model only takes as the input the question but no article about “why Einstein didn’t win a Nobel Prize for the theory of relativity” is provided, where the term “the law of the photoelectric effect” is likely mentioned. In the case when both the question and the context are provided, the task is known as Reading comprehension (RC).'

In [12]:
#faiss db

from langchain_community.vectorstores import FAISS
faissdb = FAISS.from_documents(documents[:15], OpenAIEmbeddings(openai_api_key = OPENAPIKEY))
query = "What is Open-Domain Question Answering?"
res = faissdb.similarity_search(query)
res[0].page_content

'What is Open-Domain Question Answering?#\nOpen-domain Question Answering (ODQA) is a type of language tasks, asking a model to produce answers to factoid questions in natural language. The true answer is objective, so it is simple to evaluate model performance.\nFor example,\nQuestion: What did Albert Einstein win the Nobel Prize for?\nAnswer: The law of the photoelectric effect.\nThe “open-domain” part refers to the lack of the relevant context for any arbitrarily asked factual question. In the above case, the model only takes as the input the question but no article about “why Einstein didn’t win a Nobel Prize for the theory of relativity” is provided, where the term “the law of the photoelectric effect” is likely mentioned. In the case when both the question and the context are provided, the task is known as Reading comprehension (RC).'